In [1]:
import sys
import os

# Add the absolute path of the src folder to sys.path
sys.path.append(os.path.abspath(r'C:\Users\asr12\OneDrive\Desktop\AI_traffic_management\ML\src'))

In [2]:
import os
import json
import pandas as pd
import numpy as np
from pso_algorithm import traffic_light_algorithm

# Define paths for input CSV files and output JSON file
input_folder = "C:/Users/asr12/OneDrive/Desktop/AI_traffic_management/ML/outputs/metrics"  # Root folder where video subfolders are located
output_file =  "C:/Users/asr12/OneDrive/Desktop/AI_traffic_management/ML/outputs/results/optimized_durations.json"
emergency_file = "C:/Users/asr12/OneDrive/Desktop/AI_traffic_management/ML/outputs/results/emergency.json"
# Ensure the output directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Initialize vehicle counts
vehicle_counts = []

# Function to preprocess each CSV file
def preprocess_csv(file_path):
    """Preprocess the CSV by cleaning data, handling missing values, and extracting features."""
    try:
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Check if all necessary columns exist
        required_columns = [
            'Frame',
            'Number_of_Vehicles',
            'Average_Speed_km/h',
            'Traffic_Density',
            'Queue_Length_meters',
            'Emergency_Vehicles'
        ]
        missing_columns = [col for col in required_columns if col not in df.columns]

        if missing_columns:
            print(f"Warning: Missing columns {missing_columns} in {file_path}. Skipping.")
            return None

        # Clean missing values: Fill NaNs with appropriate methods (mean, 0, or interpolation)
        df['Number_of_Vehicles'] = df['Number_of_Vehicles'].fillna(df['Number_of_Vehicles'].mean())
        df['Average_Speed_km/h'] = df['Average_Speed_km/h'].fillna(df['Average_Speed_km/h'].mean())
        df['Traffic_Density'] = df['Traffic_Density'].fillna(df['Traffic_Density'].mean())
        df['Queue_Length_meters'] = df['Queue_Length_meters'].fillna(df['Queue_Length_meters'].mean())
        df['Emergency_Vehicles'] = df['Emergency_Vehicles'].fillna("no").astype(str)

        # Feature engineering: Check for emergency vehicles
        has_emergency_vehicle = "yes" in df['Emergency_Vehicles'].values

        # Calculate average vehicle metrics
        total_vehicles = df['Number_of_Vehicles'].mean()
        
        # Return a dictionary with the required features for optimization
        return {
            'Number_of_Vehicles': total_vehicles,
            'Average_Speed_km/h': df['Average_Speed_km/h'].mean(),
            'Traffic_Density': df['Traffic_Density'].mean(),
            'Queue_Length_meters': df['Queue_Length_meters'].mean(),
            'Has_Emergency_Vehicle': has_emergency_vehicle
        }

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Traverse through all subdirectories in the metrics folder and process CSV files
for root, dirs, files in os.walk(input_folder):
    for file in sorted(files):  # Sort files to maintain consistent indexing
        if file.endswith('.csv'):  # Process only CSV files
            csv_file_path = os.path.join(root, file)
            
            # Debugging: Print the path to verify it's correct
            print(f"Processing file: {csv_file_path}")
            
            # Process and clean the CSV file
            road_data = preprocess_csv(csv_file_path)
            if road_data is not None:
                vehicle_counts.append(road_data)

# Display processed vehicle counts
print(f"Processed data for {len(vehicle_counts)} roads.")

# Ensure we have at least 4 roads to process (if fewer, add default values)
while len(vehicle_counts) < 4:
    vehicle_counts.append({
        'Number_of_Vehicles': 0,
        'Average_Speed_km/h': 0,
        'Traffic_Density': 0,
        'Queue_Length_meters': 0,
        'Has_Emergency_Vehicle': False
    })

# Assign indices as "Road 1", "Road 2", etc., and prepare data for the algorithm
vehicle_data_for_algo = [
    {
        'index': f"Road {i+1}",
        'vehicles': data['Number_of_Vehicles'],
        'emergency': data['Has_Emergency_Vehicle']
    }
    for i, data in enumerate(vehicle_counts[:4])
]

# Display the prepared data
print(vehicle_data_for_algo)

# Run the PSO-based traffic light optimization
optimized_timings = traffic_light_algorithm(vehicle_data_for_algo)

# Display results with indices
for result in optimized_timings:
    print(f"{result['index']} - Green Light: {result['green_time']} seconds, "
          f"Red Light: {result['red_time']} seconds, Priority: {result['priority']}")

for result in optimized_timings:
    for key in result:
        # Convert numpy int64 and float64 to native Python types
        if isinstance(result[key], (np.int64, np.float64)):
            result[key] = int(result[key]) if isinstance(result[key], np.int64) else float(result[key])

# Save results to JSON file
with open(output_file, "w") as file:
    json.dump(optimized_timings, file, indent=4)
print(f"Optimized timings saved to {output_file}")

with open(emergency_file, "w") as file:
    json.dump(vehicle_data_for_algo, file, indent=4)
print(f"Data of 4 roads {emergency_file}")

Processing file: C:/Users/asr12/OneDrive/Desktop/AI_traffic_management/ML/outputs/metrics\video1\video1_metrics.csv
Processing file: C:/Users/asr12/OneDrive/Desktop/AI_traffic_management/ML/outputs/metrics\video2\video2_metrics.csv
Processing file: C:/Users/asr12/OneDrive/Desktop/AI_traffic_management/ML/outputs/metrics\video3\video3_metrics.csv
Processing file: C:/Users/asr12/OneDrive/Desktop/AI_traffic_management/ML/outputs/metrics\video4\video4_metrics.csv
Processed data for 4 roads.
[{'index': 'Road 1', 'vehicles': np.float64(25.125), 'emergency': False}, {'index': 'Road 2', 'vehicles': np.float64(24.916666666666668), 'emergency': True}, {'index': 'Road 3', 'vehicles': np.float64(25.136363636363637), 'emergency': False}, {'index': 'Road 4', 'vehicles': np.float64(15.214285714285714), 'emergency': False}]
Road 2 - Green Light: 55 seconds, Red Light: 145 seconds, Priority: 1
Road 3 - Green Light: 55 seconds, Red Light: 145 seconds, Priority: 2
Road 1 - Green Light: 55 seconds, Red Li